# Reproducing Chlorophyll-a Analysis in Santa Monica Bay Using Landsat 8
This notebook aims to reproduce and extend the analysis conducted in the ocean remote sensing project titled "Remote Sensing of Chlorophyll-a using Landsat 8". The original project, available  [here](https://romero61.github.io/posts/SMB/), focused on the analysis of Chlorophyll-a concentrations in the Santa Monica Bay. The study was based on the methodology proposed by Trinh et al. 2017.

We utilize Landsat 8 satellite imagery to estimate Chlorophyll-a concentrations, and analyze changes over time, particularly focusing on the impact of the Hyperion Treatment Plant failure.

The notebook includes the following steps:

1. Importing necessary Python libraries for data manipulation, mathematical operations, data visualization, handling date and time data, interacting with Google Earth Engine, and handling geospatial data.

2. Initializing the Earth Engine API and creating an interactive map using the geemap library.

3. Defining the collection of satellite images to be used (Landsat 8 OLI images) and the study area.

4. Processing the satellite images, including applying scale factors and estimating Chlorophyll-a concentrations.

5. Visualizing the processed images on an interactive map.

The next steps of this project will include retrieving the most recent image from the collection, calculating basic statistics for Chlorophyll-a concentrations, allowing user-defined regions for analysis, and integrating this script into an R Shiny dashboard for interactive data exploration.

Trinh, R. C., Fichot, C. G., Gierach, M. M., Holt, B., Malakar, N. K., Hulley, G., & Smith, J. (2017). Application of Landsat 8 for Monitoring Impacts of Wastewater Discharge on Coastal Water Quality. Frontiers in Marine Science, 4. https://doi.org/10.3389/fmars.2017.00329

In [ ]:
#1) Import all necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
import ee
import geemap.foliumap as geemap
import geopandas as gpd
from datetime import datetime
import ipywidgets as widgets
import seaborn as sns


In [ ]:
from constants import   STUDY_BOUNDARY_PATH


In [ ]:
shapefile_path = STUDY_BOUNDARY_PATH
study_boundary = gpd.read_file(shapefile_path)

# Import Landsat8 OLI

https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2#terms-of-use

 Landsat-8 image courtesy of the U.S. Geological Survey

In [ ]:
# Initialize the Earth Engine library
#ee.Authenticate()
ee.Initialize()

In [ ]:
# Define the collection
collection = "LANDSAT/LC08/C02/T1_L2"

In [ ]:
# Load the study area
ee_boundary = geemap.geopandas_to_ee(study_boundary)


In [ ]:
aoi = ee_boundary.geometry()

In [ ]:
# Define the Image Collection
image_collection = ee.ImageCollection(collection) \
    .filterBounds(aoi) \
    .sort('system:time_start', False)

In [ ]:
# Get  ~ 2 1/2 years of images
recent_images = image_collection.limit(100)

In [ ]:
# Get the dates of the images
dates = recent_images.aggregate_array('system:time_start').getInfo()
# Convert the timestamps to readable dates
dates = [datetime.fromtimestamp(date/1000).strftime('%Y-%m-%d') for date in dates]
# Sort the list of unique dates in descending order
dates = sorted(list(set(dates)), reverse=True)

# Print the unique dates


# Print the unique dates
print("Dates of the four most recent images:", dates)

In [ ]:
# Define a function to calculate statistics
def calculate_stats(image, region):
    stats = image.reduceRegion(
        reducer=ee.Reducer.mean().combine(
            reducer2=ee.Reducer.stdDev(),
            sharedInputs=True
        ),
        geometry=region,
        scale=30,  # scale in meters
        maxPixels=1e9
    )
    return stats.getInfo()



In [ ]:
# Define a function to apply scaling and offset
def apply_scale_factors(image):
    optical_bands = image.select('SR_B.*').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    image = image.addBands(optical_bands, None, True)
    image = image.addBands(thermal_bands, None, True)
    return image


https://calekochenour.github.io/remote-sensing-textbook/03-beginner/chapter13-data-quality-bitmasks.html


In [ ]:
# Define function to return QA bands

def extract_qa_bits(qa_band, start_bit, end_bit, band_name):
    """
    Extracts QA values from an image
    :param qa_band: Single-band image of the QA layer
    :type qa_band: ee.Image
    :param start_bit: Starting bit
    :type start_bit: Integer
    :param end_bit: Ending bit
    :type end_bit: Integer
    :param band_name: New name for the band
    :type band_name: String
    :return: Image with extracted QA values
    :rtype: ee.Image
    """
    # Initialize QA bit string/pattern to check QA band against
    qa_bits = 0
    # Add each specified QA bit flag value/string/pattern to the QA bits to check/extract
    for bit in range(start_bit, end_bit + 1):
        qa_bits += pow(2, bit)  # Same as qa_bits += (1 << bit)
    # Return a single band image of the extracted QA bit values
    return qa_band.select([0], [band_name]).bitwiseAnd(qa_bits).rightShift(start_bit)


In [ ]:
# Define a function to calculate Chlorphyll-a based on trinh et al.(2017)
def trinh_et_al_chl_a(image):
    # extract the cloud and water masks
    qa_band = ee.Image(image).select('QA_PIXEL')
    cloudMask = extract_qa_bits(qa_band, 8, 9, "cloud").neq(3)  # different than 3 to remove clouds
    waterMask = extract_qa_bits(qa_band, 7, 7, "water").eq(1)  # equals 1 to keep water

    # apply the masks to the image
    image = image.updateMask(cloudMask).updateMask(waterMask)
    image =  apply_scale_factors(image)

    a_0 = 0.9375
    a_1 = -1.8862
    blue_bands = image.select('SR_B2')
    green_bands = image.select('SR_B3')
    ln_chl_a = image.expression("a_0 + a_1 * log(blue_bands/green_bands)", {
        'a_0': a_0,
        'a_1': a_1,
        'blue_bands': blue_bands,
        'green_bands': green_bands
    })
    image = image.addBands(ln_chl_a.select([0], ['ln_chl_a']))


    return image



In [ ]:
# Define a function to calculate SPM from Novoa et al.(2017) based on Nechad et al. (2010) NIR (recalibrated) model

def novoa_et_al_spm(image):
    # extract the cloud and water masks
    qa_band = ee.Image(image).select('QA_PIXEL')
    cloudMask = extract_qa_bits(qa_band, 8, 9, "cloud").neq(3)  # different than 3 to remove clouds
    waterMask = extract_qa_bits(qa_band, 7, 7, "water").eq(1)  # equals 1 to keep water

    # apply the masks to the image
    image = image.updateMask(cloudMask).updateMask(waterMask)
    image =  apply_scale_factors(image)

# Select the NIR band (B5 for Landsat 8 OLI)
    nir = image.select('SR_B5')
# Apply the Nechad et al. (2010) NIR (recalibrated) model
    spm = image.expression(
        "4302 * (nir / (1 - nir / 0.2115))",
        {'nir': nir}
    )
    # Set negative SPM values to zero
    spm = spm.where(spm.lt(0), 0)
    image = image.addBands(spm.select([0], ['spm']))


    return image

In [ ]:
# Define the function to extract the data (mean value) and valid pixel area.
def extract_data(image):
    stats = image.reduceRegion(ee.Reducer.mean(), aoi, 30)
    valid_pixels = image.select('ln_chl_a').unmask().neq(0)  # create a mask of valid pixels
    valid_area = valid_pixels.multiply(ee.Image.pixelArea()).reduceRegion(ee.Reducer.sum(), aoi, 30)  # calculate the area of valid pixels
    return image.set('date', image.date().format()).set(stats).set('Area', valid_area)

In [ ]:
def extract_data_spm(image):
    stats = image.reduceRegion(ee.Reducer.mean(), aoi, 30)
    valid_pixels = image.select('spm').unmask().neq(0)  # create a mask of valid pixels
    valid_area = valid_pixels.multiply(ee.Image.pixelArea()).reduceRegion(ee.Reducer.sum(), aoi, 30)  # calculate the area of valid pixels
    return image.set('date', image.date().format()).set(stats).set('Area', valid_area)


In [ ]:
turbo_palette = [
    "30123b", "321543", "33184a", "341b51", "351e58", "36215f", "372466", "38276d", 
    "392a73", "3a2d79", "3b2f80", "3c3286", "3d358b", "3e3891", "3f3b97", "3f3e9c", 
    "4040a2", "4143a7", "4146ac", "4249b1", "424bb5", "434eba", "4451bf", "4454c3", 
    "4456c7", "4559cb", "455ccf", "455ed3", "4661d6", "4664da", "4666dd", "4669e0", 
    "466be3", "476ee6", "4771e9", "4773eb", "4776ee", "4778f0", "477bf2", "467df4", 
    "4680f6", "4682f8", "4685fa", "4687fb", "458afc", "458cfd", "448ffe", "4391fe", 
    "4294ff", "4196ff", "4099ff", "3e9bfe", "3d9efe", "3ba0fd", "3aa3fc", "38a5fb", 
    "37a8fa", "35abf8", "33adf7", "31aff5", "2fb2f4", "2eb4f2", "2cb7f0", "2ab9ee", 
    "28bceb", "27bee9", "25c0e7", "23c3e4", "22c5e2", "20c7df", "1fc9dd", "1ecbda", 
    "1ccdd8", "1bd0d5", "1ad2d2", "1ad4d0", "19d5cd", "18d7ca", "18d9c8", "18dbc5", 
    "18ddc2", "18dec0", "18e0bd", "19e2bb", "19e3b9", "1ae4b6", "1ce6b4", "1de7b2", 
    "1fe9af", "20eaac", "22ebaa", "25eca7", "27eea4", "2aefa1", "2cf09e", "2ff19b", 
    "32f298", "35f394", "38f491", "3cf58e", "3ff68a", "43f787", "46f884", "4af880", 
    "4ef97d", "52fa7a", "55fa76", "59fb73", "5dfc6f", "61fc6c", "65fd69", "69fd66", 
    "6dfe62", "71fe5f", "75fe5c", "79fe59", "7dff56", "80ff53", "84ff51", "88ff4e", 
    "8bff4b", "8fff49", "92ff47", "96fe44", "99fe42", "9cfe40", "9ffd3f", "a1fd3d", "a4fc3c", "a7fc3a", "a9fb39", "acfb38", 
    "affa37", "b1f936", "b4f836", "b7f735", "b9f635", "bcf534", "bef434", "c1f334", 
    "c3f134", "c6f034", "c8ef34", "cbed34", "cdec34", "d0ea34", "d2e935", "d4e735", 
    "d7e535", "d9e436", "dbe236", "dde037", "dfdf37", "e1dd37", "e3db38", "e5d938", 
    "e7d739", "e9d539", "ebd339", "ecd13a", "eecf3a", "efcd3a", "f1cb3a", "f2c93a", 
    "f4c73a", "f5c53a", "f6c33a", "f7c13a", "f8be39", "f9bc39", "faba39", "fbb838", 
    "fbb637", "fcb336", "fcb136", "fdae35", "fdac34", "fea933", "fea732", "fea431", 
    "fea130", "fe9e2f", "fe9b2d", "fe992c", "fe962b", "fe932a", "fe9029", "fd8d27", 
    "fd8a26", "fc8725", "fc8423", "fb8122", "fb7e21", "fa7b1f", "f9781e", "f9751d", 
    "f8721c", "f76f1a", "f66c19", "f56918", "f46617", "f36315", "f26014", "f15d13", 
    "f05b12", "ef5811", "ed5510", "ec530f", "eb500e", "ea4e0d", "e84b0c", "e7490c", 
    "e5470b", "e4450a", "e2430a", "e14109", "df3f08", "dd3d08", "dc3b07", "da3907", 
    "d83706", "d63506", "d43305", "d23105", "d02f05", "ce2d04", "cc2b04", "ca2a04", 
    "c82803", "c52603", "c32503", "c12302", "be2102", "bc2002", "b91e02", "b71d02", 
    "b41b01", "b21a01", "af1801", "ac1701", "a91601", "a71401", "a41301", "a11201", 
    "9e1001", "9b0f01", "980e01", "950d01", "920b01", "8e0a01", "8b0902", "880802", 
    "850702", "810602", "7e0502", "7a0403"
]



In [ ]:
chloro_params= {
  'bands': ['ln_chl_a'],
  'min': 0,
  'max': 3,
  'palette': turbo_palette
}
chloro_map = geemap.Map()
chloro_map.add_basemap(basemap='TERRAIN')

spm_params= {
  'bands': ['spm'],
  'min': 0,
  'max': 50,
  'palette': 'viridis'
}
spm_map = geemap.Map()
spm_map.add_basemap(basemap='TERRAIN')

In [ ]:
processed_collection = ee.ImageCollection([])
processed_collection_spm = ee.ImageCollection([])


# Loop through the dates and get the imagery.
for date in dates:
    start_date = ee.Date(date)
    end_date = start_date.advance(1, 'day')

   # Filter the image collection by date and area
    image = ee.ImageCollection(collection) \
        .filterDate(start_date, end_date) \
        .filterBounds(aoi) \
        .first()  # get the first image that matches the filters

    if image:          # check if image exists
        
        clipped_image = image.clip(aoi)  # Clip the image to the study boundary

        processed_image = trinh_et_al_chl_a(clipped_image)  # process the image
        chloro_map.addLayer(processed_image, chloro_params, date, shown = False)  # add the image to the map
        processed_collection = processed_collection.merge(processed_image)  # add the image to the processed collection

        processed_image_spm = novoa_et_al_spm(clipped_image)  # process the image
        spm_map.addLayer(processed_image_spm, spm_params, date, shown = False)  # add the image to the map
        processed_collection_spm = processed_collection_spm.merge(processed_image_spm)  # add the image to the processed collection

    else:
        print(f"No image found for date {date}")

# Map the extract_data function over the processed collection.
extract__collection = processed_collection.map(extract_data)
# Map the extract_data function over the processed_collection_spmn.
extract__collection_spm = processed_collection_spm.map(extract_data_spm)

# Reduce the processed collection to a list of dates, chlorophyll-a values, and area.
data = extract__collection.reduceColumns(ee.Reducer.toList(3), ['date', 'ln_chl_a', 'Area']).get('list').getInfo()

data_spm = extract__collection_spm.reduceColumns(ee.Reducer.toList(3), ['date', 'spm', 'Area']).get('list').getInfo()



In [ ]:
data

In [ ]:
data_spm

In [ ]:
# Convert the data to a pandas dataframe and format the date.
df = pd.DataFrame(data, columns=['date', 'ln_chl_a', 'Area'])
df['date'] = pd.to_datetime(df['date'])
df['Area'] = df['Area'].apply(lambda x: x['ln_chl_a'])

# Add ln_chl_a_norm to the dataframe
df['ln_chl_a_norm'] = df['ln_chl_a'] / df['Area']




In [ ]:
df = df[df['date'] != '2022-05-22 18:28:16']  # Keep only the rows where 'date' is not '2022-05-22'


In [ ]:
df_spm = pd.DataFrame(data_spm, columns=['date', 'spm', 'Area'])
df_spm['date'] = pd.to_datetime(df_spm['date'])
df_spm['Area'] = df_spm['Area'].apply(lambda x: x['spm'])
df_spm['spm_norm'] = df_spm['spm'] / df_spm['Area']


In [ ]:
df

In [ ]:
df_spm


In [ ]:
df_spm = df_spm[df_spm['date'] != '2022-05-22 18:28:16']  # Keep only the rows where 'date' is not '2022-05-22'


In [ ]:
# Set the style of the plot
sns.set_style("whitegrid")

# Create the plot
plt.figure(figsize=(10, 6))
plt.plot(df['date'], df['ln_chl_a'], marker='o', linestyle='-')

# Format the x-axis to display the dates in a more readable format
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=120))  # adjust the interval as needed
plt.gcf().autofmt_xdate()  # rotate the x labels

# Add a title and labels to the axes
plt.title('Time Series of Avg Chl-a Values', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('ln(chl-a) mg/m³', fontsize=14)
# Show the plot
plt.show()

In [ ]:
#Create the plot for normalized Chlorophyll-a values
plt.figure(figsize=(10, 6))
plt.plot(df['date'], df['ln_chl_a_norm'], marker='o', linestyle='-')

#Format the x-axis as datetime
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=120)) # set interval to 10 days

#Set the labels and title
plt.xlabel('Date')
plt.ylabel('Normalized ln(Chlorophyll-a) mg/m³')
plt.title('Time series of Avg Normalized ln(Chl-a)')

#Rotate date labels automatically
plt.gcf().autofmt_xdate()

#Show the plot
plt.show()

In [ ]:
# Set the style of the plot
sns.set_style("whitegrid")

# Create the plot
plt.figure(figsize=(10, 6))
plt.plot(df_spm['date'], df_spm['spm'], marker='o', linestyle='-')

# Format the x-axis to display the dates in a more readable format
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=120))  # adjust the interval as needed
plt.gcf().autofmt_xdate()  # rotate the x labels

# Add a title and labels to the axes
plt.title('Time Series of Avg SPM Values', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('SPM g/m³', fontsize=14)
# Show the plot
plt.show()

In [ ]:
# Set the style of the plot
sns.set_style("whitegrid")

# Create the plot
plt.figure(figsize=(10, 6))
plt.plot(df_spm['date'], df_spm['spm_norm'], marker='o', linestyle='-')

# Format the x-axis to display the dates in a more readable format
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=120))  # adjust the interval as needed
plt.gcf().autofmt_xdate()  # rotate the x labels

# Add a title and labels to the axes
plt.title('Time Series of Avg Normalized SPM Values', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Normalized SPM g/m³', fontsize=14)
# Show the plot
plt.show()

# DUE TO CLOUD COVER NO IMAGES FOR 6-10, 5-25, 5-09 in 2023
# OPEN LAYER SELECTION TO VIEW 
Cloud cover leads to missing datasets and sections.

In [ ]:
# Set the map to focus on the study area
chloro_map.centerObject(aoi, zoom=11)
chloro_map.add_colorbar_branca(vis_params= chloro_params, colors = turbo_palette,vmin =  0, vmax = 3, label = 'mg/m³')
chloro_map

In [ ]:
# Set the map to focus on the study area
spm_map.centerObject(aoi, zoom=11)
spm_map.add_colorbar_branca(vis_params= spm_params, colors = turbo_palette ,vmin =  0, vmax = 50, label = 'g/m³')
spm_map

Santa Monica Bay Watershed Management Area (WMA): The Santa Monica Bay WMA encompasses an area of 414 square miles and is quite diverse. Its borders reach from the crest of the Santa Monica Mountains on the north and from the Ventura-Los Angeles County line to downtown Los Angeles. The WMA includes several watersheds, the two largest being Malibu Creek to the north (west) and Ballona Creek to the south. The Malibu Creek area contains mostly undeveloped mountain areas, large acreage residential properties and many natural stream reaches while Ballona Creek is predominantly channelized, and highly developed with both residential and commercial properties. The Santa Monica Bay was included in the National Estuary Program in 1989 and has been extensively studied by the Santa Monica Bay Restoration Project. The Santa Monica Bay Watershed Commission was established in 2002 to oversee the implementation of the Plan.

Water Quality Problems and Issues: The Santa Monica Bay WMA embraces a high diversity in geological and hydrological characteristics, habitat features, and human activities. Existing and potential beneficial use impairment problems in the watershed fall into two major categories: human health risk, and natural habitat degradation. The former are issues primarily associated with recreational uses of the Santa Monica Bay. The latter are issues associated with terrestrial, aquatic, and marine environments. Pollutant loadings that originate from human activities are common causes of both human health risks and habitat degradation.

Santa Monica Bay - Wikipedia: Santa Monica Bay is a bight of the Pacific Ocean in Southern California, United States. Its boundaries are slightly ambiguous, but it is generally considered to be the part of the Pacific within an imaginary line drawn between Point Dume, in Malibu, and the Palos Verdes Peninsula. Its eastern shore forms the western boundary of the Los Angeles Westside and South Bay regions. The Santa Monica Bay is home to some of the most famous beaches in the world, including Malibu Lagoon State Beach (Surfrider), Will Rogers State Beach, Santa Monica State Beach, and Dockweiler State Beach. Several piers extend into the bay, including Malibu Pier, Santa Monica Pier, Venice Pier, Manhattan Beach pier, Hermosa Beach pier, and Redondo Beach pier. Marina Del Rey is a dredged marina. The Bay is also a very popular fishing destination year-round. Chevron Reef is an artificial surfing reef in the bay.

Santa Monica Beach Guide: Located just west of Downtown Los Angeles, Santa Monica Beach is a prime example of the famed Southern California state beaches. With large expanses of beach, bike trails and many other, Santa Monica Beach has everything you’re looking for: Plenty of space—3.5 miles long, soft, sandy shores for little feet, walkable to hotels, shops and restaurants, offers a variety of activities and attractions.

The Santa Monica Bay, California, and the Gironde Estuary and Bourgneuf Bay in France have some similarities but also significant differences. Here are some key points to consider:

Size and Geography: The Santa Monica Bay is a bight of the Pacific Ocean, while the Gironde Estuary and Bourgneuf Bay are estuaries, which are partially enclosed coastal bodies of water where freshwater from rivers and streams meets and mixes with saltwater from the ocean. The Gironde Estuary is formed by the confluence of the Garonne and Dordogne rivers, and the Loire River feeds into Bourgneuf Bay. The Santa Monica Bay, on the other hand, is fed by several smaller watersheds, including Malibu Creek and Ballona Creek.

Turbidity and SPM Concentration: The Gironde Estuary and Bourgneuf Bay are characterized by high Suspended Particulate Matter (SPM) concentrations, ranging from 1 to 3000 g·m−3 in the Gironde and 50 to over 1000 g·m−3 in Bourgneuf Bay. The Santa Monica Bay, according to the information found, has a lower SPM concentration, with a median value of 1.3 mg/L (or approximately 0.0013 g·m−3), which is significantly lower than the French sites.

Tidal Range: Both the Gironde Estuary and Bourgneuf Bay have a macro-tidal regime, with tidal ranges from 2 to 5 m and 2 to 6 m, respectively. The tidal range in the Santa Monica Bay is not explicitly stated in the sources found, but the Pacific coast of Southern California typically experiences a smaller tidal range, usually less than 2 meters.

Freshwater Inputs: The Gironde Estuary and Bourgneuf Bay have significant freshwater inputs from large rivers, with flow rates ranging from less than 100 m3·s−1 to more than 4000 m3·s−1. The Santa Monica Bay receives freshwater inputs from several smaller watersheds, but the flow rates are likely much lower than those of the French sites.

In conclusion, while there are some similarities in terms of being coastal water bodies with freshwater inputs, the Gironde Estuary and Bourgneuf Bay in France appear to be significantly more turbid and have higher SPM concentrations than the Santa Monica Bay. Therefore, the models developed for the French sites may not be directly applicable to the Santa Monica Bay without some adjustments or recalibrations.

The Santa Monica Bay, California, and the Gironde Estuary and Bourgneuf Bay in France have some similarities but also significant differences. Here are some key points to consider:

Size and Geography: The Santa Monica Bay is a bight of the Pacific Ocean, while the Gironde Estuary and Bourgneuf Bay are estuaries, which are partially enclosed coastal bodies of water where freshwater from rivers and streams meets and mixes with saltwater from the ocean. The Gironde Estuary is formed by the confluence of the Garonne and Dordogne rivers, and the Loire River feeds into Bourgneuf Bay. The Santa Monica Bay, on the other hand, is fed by several smaller watersheds, including Malibu Creek and Ballona Creek.

Turbidity and SPM Concentration: The Gironde Estuary and Bourgneuf Bay are characterized by high Suspended Particulate Matter (SPM) concentrations, ranging from 1 to 3000 g·m−3 in the Gironde and 50 to over 1000 g·m−3 in Bourgneuf Bay. The Santa Monica Bay, according to the information found, has a lower SPM concentration, with a median value of 1.3 mg/L (or approximately 0.0013 g·m−3), which is significantly lower than the French sites.

Tidal Range: Both the Gironde Estuary and Bourgneuf Bay have a macro-tidal regime, with tidal ranges from 2 to 5 m and 2 to 6 m, respectively. The tidal range in the Santa Monica Bay is not explicitly stated in the sources found, but the Pacific coast of Southern California typically experiences a smaller tidal range, usually less than 2 meters.

Freshwater Inputs: The Gironde Estuary and Bourgneuf Bay have significant freshwater inputs from large rivers, with flow rates ranging from less than 100 m3·s−1 to more than 4000 m3·s−1. The Santa Monica Bay receives freshwater inputs from several smaller watersheds, but the flow rates are likely much lower than those of the French sites.

In conclusion, while there are some similarities in terms of being coastal water bodies with freshwater inputs, the Gironde Estuary and Bourgneuf Bay in France appear to be significantly more turbid and have higher SPM concentrations than the Santa Monica Bay. Therefore, the models developed for the French sites may not be directly applicable to the Santa Monica Bay without some adjustments or recalibrations.